In [1]:
import os
import base64
import requests
import pandas as pd

In [8]:
# Lendo os arquivos
df_docente = pd.read_csv("DM_DOCENTE.CSV")
df_aluno = pd.read_csv("DM_ALUNO.CSV")
#df_variaveis = pd.read_excel("Dicionário de Variáveis.xls", "DM_ALUNO")
df_codigo = pd.read_excel("Código_do_País_de_Nascimento_ou_Naturalização.xlsx")
df_disciplinas = pd.read_csv("dados_disciplinas.txt", sep=",")

In [10]:
df_disciplinas


,MATR_ALUNO,ANO,PERIODO,SITUACAO,MEDIA_FINAL,NUM_FALTAS,DISCIPLINA
0,1,2013,1,Aprovado,77,12,2
1,2,2013,1,Reprovado por Frequencia,0,32,1
2,3,2013,1,Reprovado por Frequencia,0,50,4
3,4,2013,2,Aprovado,60,2,2
4,4,2013,2,Aprovado,68,2,5
...,...,...,...,...,...,...,...
4637,827,2017,2,Reprovado,3,10,5
4638,828,2014,1,Reprovado por Frequencia,0,32,2
4639,829,2013,2,Reprovado por Frequencia,0,72,1
4640,830,2014,1,Reprovado por Frequencia,12,28,2
